<a href="https://colab.research.google.com/github/sofzcc/lab-sql-generation-with-transformer-api/blob/main/lab-sql-generation-with-transformer-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Generation with Transformer API

In [1]:
!pip install torch transformers bitsandbytes accelerate sqlparse

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

False

In [4]:
available_memory = torch.cuda.get_device_properties(0).total_memory

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
print(available_memory)

##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [10]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
 #   model = AutoModelForCausalLM.from_pretrained(
  #      model_name,
    #    trust_remote_code=True,
     #   torch_dtype=torch.float16,
      #  device_map="auto",
       # use_cache=True,
  #  )
#else:
    # else, load in 8 bits – this is a bit slower
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        #load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [9]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

**Original**

In [7]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [ ]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [ ]:
print(generated_sql)

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [5]:
import gc
gc.collect()

58

**Variation 1**

In [3]:
prompt_1 = """### Task
Generate a SQL query to calculate total sales revenue by product category.

### Instructions
- Use the provided database schema to construct the query.
- Ensure the query calculates revenue based on sales quantity and product price.


### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers the question.
[SQL]"""


In [ ]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question, prompt_1)

**Different Schema**

In [ ]:
prompt_2 = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE marvel_superheroes (
  hero_id INTEGER PRIMARY KEY, -- Unique ID for each superhero
  name VARCHAR(50), -- Name of the superhero
  alias VARCHAR(50), -- Superhero alias or code name
  power VARCHAR(100), -- Superpower possessed by the superhero
  city VARCHAR(50), -- City where the superhero operates
  date_of_birth DATE, -- Birthdate of the superhero
  status VARCHAR(20) -- Status of the superhero (active, retired, etc.)
);

CREATE TABLE marvel_villains (
  villain_id INTEGER PRIMARY KEY, -- Unique ID for each villain
  name VARCHAR(50), -- Name of the villain
  alias VARCHAR(50), -- Villain alias or code name
  power VARCHAR(100), -- Special abilities possessed by the villain
  nemesis VARCHAR(50) -- Arch-nemesis of the villain
);

CREATE TABLE marvel_battles (
  battle_id INTEGER PRIMARY KEY, -- Unique ID for each battle
  hero_id INTEGER, -- ID of the superhero involved
  villain_id INTEGER, -- ID of the villain involved
  location VARCHAR(50), -- Location where the battle took place
  date DATE, -- Date when the battle occurred
  outcome VARCHAR(20) -- Outcome of the battle (win, loss, draw)
);

CREATE TABLE marvel_hero_stats (
  hero_id INTEGER PRIMARY KEY, -- Unique ID for each superhero
  strength INTEGER, -- Strength level of the superhero
  agility INTEGER, -- Agility level of the superhero
  intelligence INTEGER -- Intelligence level of the superhero
);

CREATE TABLE marvel_villain_stats (
  villain_id INTEGER PRIMARY KEY, -- Unique ID for each villain
  strength INTEGER, -- Strength level of the villain
  agility INTEGER, -- Agility level of the villain
  intelligence INTEGER -- Intelligence level of the villain
);

-- marvel_battles.hero_id can be joined with marvel_superheroes.hero_id
-- marvel_battles.villain_id can be joined with marvel_villains.villain_id
-- marvel_hero_stats.hero_id can be joined with marvel_superheroes.hero_id
-- marvel_villain_stats.villain_id can be joined with marvel_villains.villain_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [ ]:
def generate_query(question, prompt):
    updated_prompt = prompt.format(question=question)

    # Tokenize the prompt
    inputs = tokenizer(updated_prompt, return_tensors="pt")

    # Generate sequence on CPU
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )

    # Decode the generated sequence
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    # Format and return the SQL query
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)


In [1]:
#For loop to generate SQL queries

questions = [
    "What is Iron Man's alias?",
    "Who is Spider-Man's arch-nemesis?",
    "List all battles involving Captain America.",
    "Which superhero has the highest strength?",
    "Which villain has the lowest intelligence?",
]

# Loop through each question and generate the corresponding SQL query
for question in questions:
    generated_sql = generate_query(question, prompt_2)
    formatted_sql = sqlparse.format(generated_sql, reindent=True)

    print(f"Question: {question}")
    print(f"Generated SQL Query:\n{formatted_sql}\n")


NameError: name 'generate_query' is not defined

**Conclusions** There isn't much to conclude as I can't run the original notebook. I need to wait until Colab allows me GPU access again.